### Twilight Struggle

In [1]:
import numpy as np
import random

from twilight_map import *
from game_mechanics import *
from twilight_cards import *

### Game Instance

In [2]:
# this is a temporary measure. It should go into UI.new()
g = Game()
g.ui.run()

Initalising game..

----- USSR Player: -----
You may modify 6 influence in these countries. Type in their country indices, separated by commas (no spaces).
Poland, 17
Finland, 7
East_Germany, 16
Romania, 21
Yugoslavia, 20
Austria, 14
Czechoslovakia, 18
Bulgaria, 22
Hungary, 19
> q

----- US Player: -----
You may modify 7 influence in these countries. Type in their country indices, separated by commas (no spaces).
France, 10
Italy, 12
Finland, 7
UK, 4
West_Germany, 15
Benelux, 9
Austria, 14
Denmark, 8
Greece, 13
Spain_Portugal, 11
Canada, 3
Sweden, 6
Turkey, 23
Norway, 5
> q

----- US Player: -----
You may modify 2 influence in these countries. Type in their country indices, separated by commas (no spaces).
US, 2
Canada, 3
UK, 4
Israel, 26
Iran, 30
Australia, 42
Philippines, 44
Japan, 45
South_Korea, 47
South_Africa, 59
Panama, 72
> q
US Victory!
Final scoring complete.
US Victory!
Final scoring complete.
US Victory!
Final scoring complete.
US Victory!
Final scoring complete.
US Victory

In [3]:
g.card_operation_add_influence(Side.USSR, 7)


----- USSR Player: -----
You may modify 7 influence in these countries. Type in their country indices, separated by commas (no spaces).
Sweden, 6
Finland, 7
Austria, 14
West_Germany, 15
East_Germany, 16
Poland, 17
Czechoslovakia, 18
Romania, 21
Turkey, 23
Israel, 26
Lebanon, 27
Syria, 28
Iraq, 29
Iran, 30
Jordan, 31
Gulf_States, 32
Saudi_Arabia, 33
Afghanistan, 34
South_Korea, 47
North_Korea, 48
Chinese_Civil_War, 87
> q
